In [1]:
class solution:
    def __init__(self):
        self.best = 0
        self.bestIndividual=[]
        self.convergence = []
        self.optimizer=""
        self.objfname=""
        self.startTime=0
        self.endTime=0
        self.executionTime=0
        self.lb=0
        self.ub=0
        self.dim=0
        self.popnum=0
        self.maxiers=0
        self.no_of_iterations=0
        self.positions=[]

In [2]:
def simplebounds(s,Lb,Ub):
    #% Apply the lower bound
    ns_tmp=np.array(s)
    I=ns_tmp<Lb
    ns_tmp[I]=Lb
  
    #% Apply the upper bounds 
    J=ns_tmp>Ub
    ns_tmp[J]=Ub
    #% Update this new move 
    return ns_tmp

In [3]:
# -*- coding: utf-8 -*-
"""
Created on Mon May 16 00:27:50 2016
@author: Hossam Faris
"""

import random
import numpy
import math
#from solution import solution
import time


    

def GWO(objf,lb,ub,dim,SearchAgents_no,Max_iter):
    
    
    #Max_iter=1000
    #lb=-100
    #ub=100
    #dim=30  
    #SearchAgents_no=
    ggnn=0
    levy=np.zeros(SearchAgents_no)
    
    # initialize alpha, beta, and delta_pos
    Alpha_pos=numpy.zeros(dim)
    Alpha_score=float("inf")
    
    Beta_pos=numpy.zeros(dim)
    Beta_score=float("inf")
    
    Delta_pos=numpy.zeros(dim)
    Delta_score=float("inf")
    
    #Initialize the positions of search agents
    Positions=numpy.random.uniform(0,1,(SearchAgents_no,dim)) *(ub-lb)+lb
    positions = numpy.random.choice([0, 1], size=(SearchAgents_no,dim))
    
    Convergence_curve=numpy.zeros(Max_iter)
    sol=solution()

     # Loop counter
    #print("GWO is optimizing  \""+objf.__name__+"\"")    
    
    timerStart=time.time() 
    sol.startTime=time.strftime("%Y-%m-%d-%H-%M-%S")
    # Main loop
    for l in range(0,Max_iter):
        for i in range(0,SearchAgents_no):
            
            # Return back the search agents that go beyond the boundaries of the search space
            Positions[i,:]=numpy.clip(Positions[i,:], lb, ub)

            # Calculate objective function for each search agent
            fitness=objf(Positions[i,:])
            
            # Update Alpha, Beta, and Delta
            if fitness<Alpha_score :
                Alpha_score=fitness; # Update alpha
                Alpha_pos=Positions[i,:].copy()
            
            
            if (fitness>Alpha_score and fitness<Beta_score ):
                Beta_score=fitness  # Update beta
                Beta_pos=Positions[i,:].copy()
            
            
            if (fitness>Alpha_score and fitness>Beta_score and fitness<Delta_score): 
                Delta_score=fitness # Update delta
                Delta_pos=Positions[i,:].copy()
            
        
        
        
        a=2-l*((2)/Max_iter); # a decreases linearly fron 2 to 0
        
        beta=0.5
        sigma=(math.gamma(1+beta)*math.sin(math.pi*beta/2)/(math.gamma((1+beta)/2)*beta*2**((beta-1)/2)))**(1/beta);
        
        # Update the Position of search agents including omegas
        for i in range(0,SearchAgents_no):
            for j in range (0,dim):     
                           
                r1=random.random() # r1 is a random number in [0,1]
                r2=random.random() # r2 is a random number in [0,1]
                
                A1=2*a*r1-a; # Equation (3.3)
                C1=2*r2; # Equation (3.4)
                
                D_alpha=abs(C1*Alpha_pos[j]-Positions[i,j]); # Equation (3.5)-part 1
                X1=Alpha_pos[j]-A1*D_alpha; # Equation (3.6)-part 1
                #else:
                 #   X1=random.random()
                #X1=numpy.random.uniform(0,1,(SearchAgents_no,dim)) *(ub-lb)+lb
                           
                r1=random.random()
                r2=random.random()
                
                A2=2*a*r1-a; # Equation (3.3)
                C2=2*r2; # Equation (3.4)
                
                D_beta=abs(C2*Beta_pos[j]-Positions[i,j]); # Equation (3.5)-part 2
                #if j%2==1:
                X2=Beta_pos[j]-A2*D_beta; # Equation (3.6)-part 2
                #else:
                #    X2=random.random()
                
                r1=random.random()
                r2=random.random() 
                
                A3=2*a*r1-a; # Equation (3.3)
                C3=2*r2; # Equation (3.4)
                
                D_delta=abs(C3*Delta_pos[j]-Positions[i,j]); # Equation (3.5)-part 3
                X3=Delta_pos[j]-A3*D_delta; # Equation (3.5)-part 3             
                
                Positions[i,j]=(X1+X2+X3)/3  # Equation (3.7)
                #rr=levy(,j)
                
            
            #ggnn=alpha(i,Max_iter)
            s=Positions[i,:]
            
            u=numpy.random.uniform(size=len(s))*sigma
            v=numpy.random.uniform(size=len(s))
            step=u*abs(v)**(1/beta)
            stepsize=0.1*step*(s-Alpha_pos) #s-alpha_pos
            s=s+stepsize*numpy.random.uniform(size=len(s))
            Positions[i]=simplebounds(s,lb,ub)
            rand=random.random()
            if(sigmoid(Positions[i-1,j])>=rand):
                positions[i,j]=1

            else:
                positions[i,j]=0
        Convergence_curve[l]=Alpha_score
        """if Convergence_curve[l]==Convergence_curve[l-1]==Convergence_curve[l-2]==Convergence_curve[l-3]==Convergence_curve[l-4]==Convergence_curve[l-5]==Convergence_curve[l-6]:
            print(Convergence_curve[l],Convergence_curve[l-6])
            break"""

        """if (l%1==0):
               print(['At iteration '+ str(l)+ ' the best fitness is '+ str(Alpha_score)]);"""
    
    timerEnd=time.time()  
    sol.endTime=time.strftime("%Y-%m-%d-%H-%M-%S")
    sol.executionTime=timerEnd-timerStart
    sol.convergence=Convergence_curve
    sol.optimizer="GWO"
    sol.objfname=objf.__name__
    sol.no_of_iterations=l
    sol.positions=positions
    
    
    return sol
  
  

In [4]:
import math

def sigmoid(x):
    return 1 / (1 + math.exp(-x))

In [9]:
def F1(x):
    s=numpy.sum(x**2);
    return s

In [8]:
import numpy as np
import pandas as pd

In [10]:
liwc=pd.read_csv("LIWC2015 Results (twitter-sanders-apple3).csv")

In [11]:
liwc["Source (A)"].unique()

array(['class', 'Pos', 'Neg', 'Neutral'], dtype=object)

In [12]:
ynew=liwc["Source (A)"].values

In [71]:
liwcnew=applyBinary(liwc)

TypeError: applyBinary() missing 1 required positional argument: 'newnewPos'

In [160]:
liwcnew.head(10)

,Source (A),Source (B),WC,Analytic,Authentic,Sixltr,function,you,shehe,they,...,netspeak,assent,filler,AllPunc,SemiC,QMark,Exclam,Dash,Apostro,OtherP
0,class,text,1,92.84,1.00,0.00,0.00,0.00,0.0,0.0,...,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.00,0.00
1,Pos,Now all @Apple has to do is get swype on the i...,20,15.01,3.37,0.00,65.00,0.00,0.0,0.0,...,0.00,0.0,0.0,10.00,0.0,0.00,0.00,0.00,0.00,5.00
2,Pos,@Apple will be adding more carrier support to ...,13,82.00,1.40,23.08,38.46,0.00,0.0,0.0,...,0.00,0.0,0.0,23.08,0.0,0.00,0.00,0.00,0.00,7.69
3,Pos,Hilarious @youtube video - guy does a duet wit...,23,99.00,1.00,13.04,21.74,0.00,0.0,0.0,...,4.35,0.0,0.0,39.13,0.0,0.00,4.35,4.35,4.35,17.39
4,Pos,@RIM you made it too easy for me to switch to ...,15,69.57,1.00,0.00,53.33,13.33,0.0,0.0,...,6.67,0.0,0.0,26.67,0.0,0.00,6.67,0.00,0.00,13.33
5,Pos,I just realized that the reason I got into twi...,14,47.35,99.00,14.29,50.00,0.00,0.0,0.0,...,7.14,0.0,0.0,7.14,0.0,0.00,0.00,0.00,0.00,7.14
6,Pos,I'm a current @Blackberry user little bit disa...,18,60.80,87.07,22.22,44.44,0.00,0.0,0.0,...,0.00,0.0,0.0,33.33,0.0,0.00,5.56,0.00,5.56,22.22
7,Pos,The 16 strangest things Siri has said so far. ...,28,67.14,13.15,10.71,39.29,0.00,0.0,0.0,...,7.14,0.0,0.0,25.00,0.0,0.00,3.57,0.00,0.00,14.29
8,Pos,Great up close & personal event @Apple tonight...,11,99.00,99.00,18.18,18.18,0.00,0.0,0.0,...,0.00,0.0,0.0,27.27,0.0,0.00,9.09,0.00,0.00,18.18
9,Pos,From which companies do you experience the bes...,15,83.85,1.00,26.67,46.67,6.67,0.0,0.0,...,0.00,0.0,0.0,20.00,0.0,6.67,0.00,0.00,0.00,13.33


In [166]:
liwc_all=liwc.copy()
#new.drop('time',axis=1,inplace=True)
liwc_all.drop("Source (A)",axis=1,inplace=True)
liwc_all.drop("Source (B)",axis=1,inplace=True)
liwcnew.drop("Source (A)",axis=1,inplace=True)
liwcnew.drop("Source (B)",axis=1,inplace=True)

E:\Anaconda3\envs\tensorflow\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [167]:
liwc_all.head(10)

,WC,Analytic,Clout,Authentic,Tone,WPS,Sixltr,Dic,function,pronoun,...,Comma,Colon,SemiC,QMark,Exclam,Dash,Quote,Apostro,Parenth,OtherP
0,1,92.84,50.00,1.00,25.77,1.00,0.00,0.00,0.00,0.00,...,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.00,0.00,0.00
1,20,15.01,50.00,3.37,25.77,10.00,0.00,75.00,65.00,10.00,...,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.00,0.00,5.00
2,13,82.00,50.00,1.40,99.00,13.00,23.08,69.23,38.46,0.00,...,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.00,15.38,7.69
3,23,99.00,90.38,1.00,99.00,7.67,13.04,60.87,21.74,0.00,...,0.0,0.0,0.0,0.00,4.35,4.35,0.0,4.35,0.00,17.39
4,15,69.57,97.69,1.00,99.00,7.50,0.00,73.33,53.33,26.67,...,0.0,0.0,0.0,0.00,6.67,0.00,0.0,0.00,0.00,13.33
5,14,47.35,23.75,99.00,99.00,14.00,14.29,85.71,50.00,21.43,...,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.00,0.00,7.14
6,18,60.80,4.80,87.07,1.00,9.00,22.22,72.22,44.44,16.67,...,0.0,0.0,0.0,0.00,5.56,0.00,0.0,5.56,0.00,22.22
7,28,67.14,63.97,13.15,87.20,9.33,10.71,67.86,39.29,10.71,...,0.0,0.0,0.0,0.00,3.57,0.00,0.0,0.00,0.00,14.29
8,11,99.00,81.84,99.00,99.00,11.00,18.18,72.73,18.18,0.00,...,0.0,0.0,0.0,0.00,9.09,0.00,0.0,0.00,0.00,18.18
9,15,83.85,90.87,1.00,99.00,15.00,26.67,80.00,46.67,13.33,...,0.0,0.0,0.0,6.67,0.00,0.00,0.0,0.00,0.00,13.33


In [168]:
vals=liwcnew.values

In [122]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier


clf=SVC(kernel='poly',degree=2,C=0.01)
clf6=KNeighborsClassifier(n_neighbors=3)
clf2=LogisticRegression(C=2)

def findPositionArray(dimension):
    egwo=GWO(objf=F1,Max_iter=1000,lb=-100,ub=100,dim=dimension,SearchAgents_no=5)
    egwo.no_of_iterations,egwo.convergence[len(egwo.convergence)-1]
    newPos=egwo.positions[0]
    newnewPos=[]
    for i in range(len(egwo.positions[0])):
        if newPos[i]==0:
            newnewPos.append(False)
        else:
            newnewPos.append(True)
    return newnewPos

def applyBinary(data,newnewPos):
    datanew=data.loc[:,newnewPos]
    return datanew

def function(dataset,clf=clf,clf6=clf6):
    #from sklearn.datasets import load_iris
    #iris=load_iris()
    allData=pd.DataFrame(dataset.data)

    newnewPos=findPositionArray(len(dataset.data[0]))
    allnew=applyBinary(allData,newnewPos)

    x_train,x_test,y_train,y_test=train_test_split(allnew.values,dataset.target,random_state=0)
    x_train_all,x_test_all,y_train_all,y_test_all=train_test_split(dataset.data,dataset.target,random_state=0)

    print("svm")
    clf.fit(x_train,y_train)
    ypredsvm=clf.predict(x_test)
    print("selected:",clf.score(x_train,y_train),clf.score(x_test,y_test))

    clf.fit(x_train_all,y_train_all)
    ypredsvm=clf.predict(x_test_all)
    print("all     :",clf.score(x_train_all,y_train_all),clf.score(x_test_all,y_test_all))

    print("knn")
    clf6.fit(x_train,y_train)
    ypredKNN=clf6.predict(x_test)
    print("selected:",clf6.score(x_train,y_train),clf6.score(x_test,y_test))

    clf6.fit(x_train_all,y_train_all)
    ypredKNN=clf6.predict(x_test_all)
    print("all     :",clf6.score(x_train_all,y_train_all),clf6.score(x_test_all,y_test_all))

def binary(ion,no,clf=clf,clf6=clf6):
    output=ion[ion.columns[no]].values
    del ion[ion.columns[no]]
    newnewPos=findPositionArray(len(ion.columns))
    allnew=applyBinary(ion,newnewPos)
    x_train,x_test,y_train,y_test=train_test_split(allnew.values,output,random_state=0)
    x_train_all,x_test_all,y_train_all,y_test_all=train_test_split(ion.values,output,random_state=0)

    print("svm")
    clf.fit(x_train,y_train)
    ypredsvm=clf.predict(x_test)
    print("selected:",clf.score(x_train,y_train),clf.score(x_test,y_test))

    clf.fit(x_train_all,y_train_all)
    ypredsvm=clf.predict(x_test_all)
    print("all     :",clf.score(x_train_all,y_train_all),clf.score(x_test_all,y_test_all))

    print("knn")
    clf6.fit(x_train,y_train)
    ypredKNN=clf6.predict(x_test)
    print("selected:",clf6.score(x_train,y_train),clf6.score(x_test,y_test))

    clf6.fit(x_train_all,y_train_all)
    ypredKNN=clf6.predict(x_test_all)
    print("all     :",clf6.score(x_train_all,y_train_all),clf6.score(x_test_all,y_test_all))

In [78]:
from sklearn.datasets import load_breast_cancer
breast=load_breast_cancer()
function(breast,clf,clf6)

svm
selected: 0.9577464788732394 0.9370629370629371


E:\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
E:\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


all     : 0.9812206572769953 0.958041958041958
knn
selected: 0.9295774647887324 0.916083916083916
all     : 0.9577464788732394 0.9230769230769231


In [123]:
from sklearn.datasets import load_iris
iris=load_iris()
function(iris)

svm
selected: 0.9553571428571429 0.9473684210526315
all     : 0.9732142857142857 0.9736842105263158
knn
selected: 0.9642857142857143 0.8947368421052632
all     : 0.9642857142857143 0.9736842105263158


E:\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
E:\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [124]:
from sklearn.datasets import load_wine
wine=load_wine()
function(wine)

svm


E:\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


selected: 1.0 0.9333333333333333
all     : 1.0 0.9777777777777777
knn
selected: 0.8721804511278195 0.7555555555555555
all     : 0.8872180451127819 0.7333333333333333


E:\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [125]:
from sklearn.datasets import load_digits
digits=load_digits()
#print(digits.DESCR)
function(digits)

svm
selected: 0.9881217520415738 0.9377777777777778


E:\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
E:\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


all     : 1.0 0.9822222222222222
knn
selected: 0.9584261321455085 0.9177777777777778
all     : 0.991833704528582 0.9866666666666667


In [126]:
ion=pd.read_csv("ionosphere.data.csv")
binary(ion,34)

svm
selected: 0.6412213740458015 0.6363636363636364
all     : 0.6412213740458015 0.6363636363636364
knn
selected: 0.916030534351145 0.8522727272727273
all     : 0.8893129770992366 0.9204545454545454


E:\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
E:\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [127]:
sonar=pd.read_csv("sonar.all-data.csv")
binary(sonar,60)

svm
selected: 0.5419354838709678 0.5192307692307693
all     : 0.5419354838709678 0.5192307692307693
knn
selected: 0.9290322580645162 0.8076923076923077
all     : 0.9096774193548387 0.8076923076923077


E:\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
E:\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
